In [1]:
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import time
from datetime import datetime
import os

# prod
user = 'read_only'
pwd = 'WHC63fKyjudfSz2U'
host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
db_name = 'CAPC_APIGATEWAY'
env='production'


# # stag
# user = 'read_only'
# pwd = '7TYtBma28775c4Ua'
# host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# db_name = 'CAPC_APIGATEWAY'
# env='staging'


db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(user,pwd,host,db_name)
db_connection = create_engine(db_connection_str)

In [11]:

acc=[]
for i in open('/home/trishalaswain/ClientData/Other/Accounts.txt','r').readlines():
    acc.append(i.rstrip('\n'))

In [12]:
len(acc)

8419

In [16]:
df_all=[]

# for f_id in type_id:
# query ='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     dm.id as document_id,
#     em.id as encounter_id
# FROM
#     account_mst am
#         INNER JOIN
#     document_mst dm ON dm.account_id = am.id
#         INNER JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON dm.account_id = em.account_id
#         AND dm.service_date = em.service_date
#         AND dm.physician_id = em.physician_id
#         AND em.is_active = 1
# WHERE
#         is_physician_document = 1
#         AND dm.is_active = 1
#         and account_number in ({})
#         and dpd.id = (select MAX(id)FROM document_processing_detail
#           WHERE document_id = dm.id)

# ORDER BY dm.created_date DESC;
# '''.format(','.join(['"'+str(x)+'"' for x in acc]))
# print(query)

# for f in fac:

# query='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         left JOIN
#     document_code dc ON dc.document_id = dm.id
#         left JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     dc.code_category = 'EANDM'
#         and is_physician_document=1
#         and dm.is_active=1
#         and account_number in ({})
#         and dpd.id = (SELECT MAX(id)
#         FROM document_processing_detail
#         WHERE document_id = dm.id)
#     order by dm.created_date desc;
# '''.format(','.join(str(x) for x in acc))

# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id AS encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         LEFT JOIN
#     document_code dc ON dc.document_id = dm.id
#         LEFT JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#         dc.code_category = 'EANDM'
#         is_physician_document = 1
#         AND am.account_number in ({})
#         AND dm.is_active = 1
#         AND am.is_active = 1
# ORDER BY dpd.created_date DESC;'''.format(','.join(str(x) for x in acc))

           
# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id AS encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         LEFT JOIN
#     document_code dc ON dc.document_id = dm.id
#         LEFT JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     am.facility_id = 38
#         AND is_physician_document = 1
#         AND dm.service_date >= '2023-05-01'
#         AND dm.is_active = 1
#         AND am.is_active = 1
# ORDER BY dpd.created_date DESC
# '''

query='''
SELECT DISTINCT
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    em.id as encounter_id,
    dc.code AS suggested_code,
    dpd.id AS DPD_ID,
    dm.path as filePath,
    ec.code AS final_code
FROM
    account_mst am
        JOIN
    document_mst dm ON dm.account_id = am.id
        JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    encounter_mst em ON am.id = em.account_id
        left JOIN
    document_code dc ON dc.document_id = dm.id
        left JOIN
    eandm_code ec ON ec.encounter_id = em.id
WHERE
        -- dc.code_category = 'EANDM'
        is_physician_document=1
        and dm.is_active=1
        and em.id in ({})
        and dpd.id = (SELECT MAX(id)
        FROM document_processing_detail
        WHERE document_id = dm.id)
    order by dm.created_date desc;
'''.format(','.join(['"'+str(x)+'"' for x in acc]))



In [17]:
# df_all

In [18]:
# df_query=pd.concat(df_all)
df_query = pd.read_sql(query, con=db_connection)

In [19]:
df_query

,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,type_id,...,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,encounter_id,suggested_code,DPD_ID,filePath,final_code
0,8842335,ezcapc/CHS/data/1475/nlp/cdp-xml/20230711/1065...,ezcapc/CHS/data/1475/nlp/ner/20230711/10650071...,90,2023-07-11 20:45:00,ezcapc/CHS/data/1475/nlp/pcs-xml/20230711/1065...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,None,27658,...,ezcapc/CHS/data/1475/eandmcs/20230711/9971621_...,ezcapc/CHS/data/1475/ruleengine/20230711/68129...,ezcapc/CHS/data/1475/cptcs/20230711/10450051_D...,ezcapc/CHS/data/1475/cptcs/20230711/10450051_D...,8479681,8339883,J45.40,10728838,CHS/data/1475/document/20230711/DS4149212CE147...,99213
1,8842335,ezcapc/CHS/data/1475/nlp/cdp-xml/20230711/1065...,ezcapc/CHS/data/1475/nlp/ner/20230711/10650071...,90,2023-07-11 20:45:00,ezcapc/CHS/data/1475/nlp/pcs-xml/20230711/1065...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,None,27658,...,ezcapc/CHS/data/1475/eandmcs/20230711/9971621_...,ezcapc/CHS/data/1475/ruleengine/20230711/68129...,ezcapc/CHS/data/1475/cptcs/20230711/10450051_D...,ezcapc/CHS/data/1475/cptcs/20230711/10450051_D...,8479681,8339883,95117,10728838,CHS/data/1475/document/20230711/DS4149212CE147...,99213
2,8842335,ezcapc/CHS/data/1475/nlp/cdp-xml/20230711/1065...,ezcapc/CHS/data/1475/nlp/ner/20230711/10650071...,90,2023-07-11 20:45:00,ezcapc/CHS/data/1475/nlp/pcs-xml/20230711/1065...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,None,27658,...,ezcapc/CHS/data/1475/eandmcs/20230711/9971621_...,ezcapc/CHS/data/1475/ruleengine/20230711/68129...,ezcapc/CHS/data/1475/cptcs/20230711/10450051_D...,ezcapc/CHS/data/1475/cptcs/20230711/10450051_D...,8479681,8339883,99213,10728838,CHS/data/1475/document/20230711/DS4149212CE147...,99213
3,8684934,ezcapc/CHS/data/1475/nlp/cdp-xml/20230711/1065...,ezcapc/CHS/data/1475/nlp/ner/20230711/10650016...,90,2023-07-11 16:15:00,ezcapc/CHS/data/1475/nlp/pcs-xml/20230711/1065...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,None,27658,...,ezcapc/CHS/data/1475/eandmcs/20230711/9971593_...,ezcapc/CHS/data/1475/ruleengine/20230711/68129...,ezcapc/CHS/data/1475/cptcs/20230711/10450022_D...,ezcapc/CHS/data/1475/cptcs/20230711/10450022_D...,10044215,8339856,T63.421D,10728806,CHS/data/1475/document/20230711/DS4148975CE147...,99212
4,8684934,ezcapc/CHS/data/1475/nlp/cdp-xml/20230711/1065...,ezcapc/CHS/data/1475/nlp/ner/20230711/10650016...,90,2023-07-11 16:15:00,ezcapc/CHS/data/1475/nlp/pcs-xml/20230711/1065...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,ezcapc/CHS/data/1475/preprocessing/20230711/10...,None,27658,...,ezcapc/CHS/data/1475/eandmcs/20230711/9971593_...,ezcapc/CHS/data/1475/ruleengine/20230711/68129...,ezcapc/CHS/data/1475/cptcs/20230711/10450022_D...,ezcapc/CHS/data/1475/cptcs/20230711/10450022_D...,10044215,8339856,95117,10728806,CHS/data/1475/document/20230711/DS4148975CE147...,99212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78093,4922264,ezcapc/CHS/data/16432/nlp/cdp-xml/20230413/784...,ezcapc/CHS/data/16432/nlp/ner/20230413/7843765...,234,2023-04-13 21:00:00,ezcapc/CHS/data/16432/nlp/pcs-xml/20230413/784...,ezcapc/CHS/data/16432/preprocessing/20230413/7...,ezcapc/CHS/data/16432/preprocessing/20230413/7...,None,69680,...,ezcapc/CHS/data/16432/eandmcs/20230413/7495765...,ezcapc/CHS/data/16432/ruleengine/20230413/4655...,ezcapc/CHS/data/16432/cptcs/20230413/7827510_D...,ezcapc/CHS/data/16432/cptcs/20230413/7827510_D...,7658611,6334541,Z12.4,7939301,CHS/data/16432/document/20230413/DS772722CE164...,99395
78094,4922264,ezcapc/CHS/data

In [20]:
df_query.account_number=df_query.account_number.apply(lambda x:str(x))

In [21]:
df_query=df_query.groupby(["filePath", "service_date", 'facility_id', 'type_id']).first().reset_index()

In [22]:
df_query=df_query.sort_values(['facility_id', 'type_id'])

In [23]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'account_number',
       'nlp_request_s3_path', 'nlp_ner_entities_result_s3_path',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'created_date',
       'ocr_request_s3_path', 'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id',
       'encounter_id', 'suggested_code', 'DPD_ID', 'final_code'],
      dtype='object')

In [24]:
len(df_query)

8419

In [25]:
print(df_query.groupby('facility_id').size().to_markdown())

|   facility_id |    0 |
|--------------:|-----:|
|            17 |  253 |
|            19 |  140 |
|            89 | 1735 |
|            90 | 1004 |
|           197 |  300 |
|           198 |  316 |
|           199 |  130 |
|           206 | 1591 |
|           207 |   32 |
|           210 |  113 |
|           211 |  179 |
|           213 |   77 |
|           218 |  166 |
|           227 |  163 |
|           230 |  214 |
|           233 |  527 |
|           234 |  527 |
|           235 |  413 |
|           239 |  409 |
|           252 |  130 |


In [26]:
# commands

In [27]:
def parse_and_create_script(x):
    print(x.facility_id.unique())
    
    df_query=x
    cmd=[]
    fac_list=[]

    for pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id,suggested_code, final_code, dpd_id,t_id, file_path in zip(df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                          df_query.nlp_pcs_xml_result_s3_path,
                                                             df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                             df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                             df_query.account_id, df_query.type_id, 
                                                             df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                             df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                             df_query.ruleengine_request_s3_path,
                                                             df_query.encounter_id,
                                                             df_query.suggested_code, df_query.final_code,
                                                             df_query.DPD_ID,
                                                             df_query.type_id,
                                                             df_query.filePath
                                                            ):
        if pcs == None or acc==None or em_req==None:
            continue
        fac_name=file_path.split("/")[2]
        cli=file_path.split("/")[0]
        fac_list.append(fac_name)

        # eandm
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_{}_pcs.xml".format(env, pcs, fac_name,enc_id,enc_id, acc, suggested_code, final_code))
        print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_{}_eandm_request.json".format(env, em_req, fac_name, enc_id,enc_id, acc, suggested_code, final_code))
        cmd.append((pcs, '{}/{}/{}_{}_{}_{}_pcs.xml'.format(fac_name, enc_id,enc_id, acc, suggested_code, final_code)))
        cmd.append((em_req, '{}/{}/{}_{}_{}_{}_eandm_req.json'.format(fac_name, enc_id,enc_id, acc, suggested_code, final_code)))
        
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, em_res,fac_name, acc_id, acc))
        # cmd.append((em_res, '{}/{}/{}_{}_eandm_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # cpt
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, dpd_id, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_cpt_request.json".format(env, cpt_req,fac_name, enc_id, dpd_id,t_id))

        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))
        # cmd.append((cpt_req, '{}/{}/{}_{}_cpt_req.json'.format(fac_name,dpd_id, enc_id,acc)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, ctp_res,fac_name, acc_id, acc))

        # preprocess
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))

        # original path
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}_{}_{}_{}.pdf".format(env,ori_path,fac_name, enc_id, t_id, acc, suggested_code, final_code))
        # cmd.append((ori_path, '{}/{}/{}_{}_{}_{}.pdf'.format(fac_name, enc_id, t_id, acc, suggested_code, final_code)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

        # ocr
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))

        # nlp
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPrequest.json".format(env, nlp_req,fac_name,enc_id, acc, suggested_code))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_NLPresult.json".format(env, nlp_res,fac_name, enc_id, acc))
        # cmd.append((nlp_req, '{}/{}/{}_{}_nlp_req.json'.format(fac_name,dpd_id, acc, t_id)))
        # cmd.append((nlp_res, '{}/{}/{}_{}_nlp_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # rule
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

        # print(pre)

    cmd.sort()
    commands=[]

    for c in cmd:
        temp=c[0].rsplit('/', 1)
        # print(temp)
        if len(commands) == 0:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
        elif temp[0] in commands[-1]:
            if 'exclude' in commands[-1]:
                commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
        else:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
        # print('-----------------------------------------')
        # print(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    return cmd, commands, fac_name

In [28]:
def print_downlaod_files(commands,cmd, fac):
    print(len(commands))
    time_now=datetime.now()
    time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

    file=open("/home/trishalaswain/ClientData/CHS/CHS_JULY/{}_docs_download_{}.sh".format(fac,time_now),'w')
    for f in commands:
        file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + re.sub('_\w+.\d{3}_', '*',f) +'\n')
        # print('===================')
        # print(f)
    # file.write('sh {}_name_change_*'.format(fac))
    
    for f in cmd:
        # print(f)
        f=(f[0].rsplit('/', 1), f[1])
        # print('mv "'+f[0][1] +'" ' +f[1])
        file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
    file.write('tar -cJf docs_{}_JULY.tar.xz {}\n'.format(fac, fac))
    file.write('rm -r {}'.format(fac))
    file.close()


In [29]:
# def print_filename_change(cmd, fac):
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     file=open("/home/devanshparmar/devansh/Download/{}_name_change_{}.sh".format(fac,time_now),'w')
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
#     file.write('rm -r {}'.format(fac))
#     file.close()

In [30]:
def do_all(x):
    # print(len(commands))
    # for cmd in commands:
    # print(commands[0], commands[-1])
    
    cmd, commands, fac = parse_and_create_script(x)
    print_downlaod_files(commands, cmd, fac)
    # print_filename_change(cmd, fac)


In [31]:
df_query.groupby('facility_id').apply(lambda x : do_all(x))

[17]
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/2839/nlp/pcs-xml/20230523/10498460_DS447896CE2839_20230706172622.022_result.xml' 2839/7242947/7242947_2424322_S01.01XA_99211_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/2839/eandmcs/20230523/9836686_DS447896CE2839_20230706172651.051_request.json' 2839/7242947/7242947_2424322_S01.01XA_99211_eandm_request.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/2839/nlp/pcs-xml/20230618/10465128_DS452077CE2839_20230705184326.026_result.xml' 2839/7822218/7822218_2431420_Z68.34_99203_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/2839/eandmcs/20230618/9806662_DS452077CE2839_20230705184555.055_request.json' 2839/7822218/7822218_2431420_Z68.34_99203_eandm_request.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/2839/nlp/pcs-xml/20230618/10328481_DS452093CE2839_20230630180423.023_result.xml' 2839/8115027/8115027_2431417_Z11.1_99211_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/

aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/9954/eandmcs/20230707/9868919_DS3093987CE9954_20230707163619.019_request.json' 9954/8260227/8260227_10973896_M17.0_99212_eandm_request.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/9954/nlp/pcs-xml/20230707/10535269_DS3093996CE9954_20230707162949.049_result.xml' 9954/8260731/8260731_10973908_M25.531_99212_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/9954/eandmcs/20230707/9870320_DS3093996CE9954_20230707170048.048_request.json' 9954/8260731/8260731_10973908_M25.531_99212_eandm_request.json
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/9954/nlp/pcs-xml/20230707/10534111_DS3094002CE9954_20230707160940.040_result.xml' 9954/8260333/8260333_13097764_M25.511_99213_pcs.xml
aws s3 cp 's3://ezdi-production-bucket/ezcapc/CHS/data/9954/eandmcs/20230707/9869182_DS3094002CE9954_20230707164057.057_request.json' 9954/8260333/8260333_13097764_M25.511_99213_eandm_request.json
aws s3 cp 's3://ezdi-production-b

""


In [32]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'account_number',
       'nlp_request_s3_path', 'nlp_ner_entities_result_s3_path',
       'nlp_pcs_xml_result_s3_path', 'preprocessing_request_s3_path',
       'preprocessing_result_s3_path', 'orignal_path', 'created_date',
       'ocr_request_s3_path', 'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id',
       'encounter_id', 'suggested_code', 'DPD_ID', 'final_code'],
      dtype='object')

In [21]:
# df_query[df_query.account_number.str.contains('0996')]

In [28]:
# f=open("/home/devanshparmar/devansh/Download/docs_chs.sh".format(cli,fac_name),'a')
# for d in cmd:
#     f.write(d+'\n')
    
# f.write('tar -cJf docs.tar.xz *\n')
# f.write('echo done\n')
# f.close()